In [1]:
import pandas as pd
import os
import time
import pyodbc
from datetime import datetime

def ensure_file_available(file_path):
    if not os.path.exists(file_path):
        print("File is not available. Please check your file in one drive")
        while not os.path.exists(file_path):
            time.sleep(5)
    print("File is available. Proceeding the processes")

def clean_and_groupby_file_PO_pending():
    file_path = r'C:\Users\Thanawit C\OneDrive - Sahamit Product Co.,Ltd\Data for Stock Report\COPY_Summary Forecast.xlsx'
    save_path = r'D:\Data for Stock Report\cleaned_PO_pending_import_party.xlsx'
    
    ensure_file_available(file_path)

    try:
        # Attempt to read the Excel file
        df2 = pd.read_excel(file_path, sheet_name='Summary Forecast', header=3)
    except PermissionError:
        print(f"Permission denied: {file_path}. Please check if the file is open or if you have access.")
        return
    except FileNotFoundError:
        print(f"File not found: {file_path}. Please check the file path.")
        return
    except Exception as e:
        print(f"An error occurred while reading the file: {e}")
        return

    # Convert the specific columns to datetime
    df2['แผนส่งเข้าคลังโพธาราม'] = pd.to_datetime(df2['แผนส่งเข้าคลังโพธาราม'], errors='coerce')
    df2['แผนส่งเข้าคลังบางปะกง'] = pd.to_datetime(df2['แผนส่งเข้าคลังบางปะกง'], errors='coerce')
    df2['แผนส่งเข้าคลังขอนแก่น'] = pd.to_datetime(df2['แผนส่งเข้าคลังขอนแก่น'], errors ='coerce')

    today = pd.to_datetime(datetime.now().date())
    
    # Filter based on the condition that either column F or H has a date greater than today
    df2_filtered = df2[(df2['แผนส่งเข้าคลังโพธาราม'] >= today) | (df2['แผนส่งเข้าคลังบางปะกง'] >= today) | (df2['แผนส่งเข้าคลังขอนแก่น'] >= today)]
    
    # Select needed columns
    selected_columns = df2_filtered[['SHM_Item',
                                      'CJ_Item',
                                      'Name',
                                      'PO CJ No.',
                                      'เรือ ETA',
                                      'แผนส่งเข้าคลังโพธาราม',
                                      'ยอดสั่งโพธาราม(ลัง)',
                                      'แผนส่งเข้าคลังบางปะกง',
                                      'ยอดสั่งบางปะกง(ลัง)',
                                      'แผนส่งเข้าคลังขอนแก่น',
                                      'ยอดสั่งขอนแก่น(ลัง)']]
    
    # Rename columns
    selected_columns.rename(columns={'Name' : 'Product Name',
                                     'แผนส่งเข้าคลังโพธาราม': 'Date_to_DC1',
                                     'ยอดสั่งโพธาราม(ลัง)': 'cartons_to_DC1',
                                     'แผนส่งเข้าคลังบางปะกง': 'Date_to_DC2',
                                     'ยอดสั่งบางปะกง(ลัง)': 'cartons_to_DC2',
                                     'แผนส่งเข้าคลังขอนแก่น': 'Date_to_DC4',
                                     'ยอดสั่งขอนแก่น(ลัง)': 'cartons_to_DC4'}, inplace=True)
    
    
    # Step to connect to Access database and load data
    access_db_path = r'D:\DataBase Access\SHM_TMS_001_Master_Copy.accdb'  # Updated Access DB path
    connection_string = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};DBQ={access_db_path};"

    conn = None
    
    try:
        # Connect to Access database
        conn = pyodbc.connect(connection_string)
        
        # Load data from Access
        query = """
        SELECT SHM_Item,
               CJ_Item,
               PC_Cartons
        FROM qry_Product_List
        """

        access_data = pd.read_sql(query, conn)


        # Merge Access data with the selected columns
        cleaned_data = pd.merge(selected_columns, access_data[['CJ_Item', 'SHM_Item', 'PC_Cartons']], on=['CJ_Item', 'SHM_Item'], how='left')


        # Calculate new columns
        cleaned_data['PO_Qty_to_DC1'] = cleaned_data['cartons_to_DC1'] * cleaned_data['PC_Cartons']
        cleaned_data['PO_Qty_to_DC2'] = cleaned_data['cartons_to_DC2'] * cleaned_data['PC_Cartons']
        cleaned_data['PO_Qty_to_DC4'] = cleaned_data['cartons_to_DC4'] * cleaned_data['PC_Cartons']



        # Group the cleaned data for the pivot table
        pivot_data = cleaned_data.groupby(['SHM_Item', 'CJ_Item']).agg({
            'PO_Qty_to_DC1': 'sum',
            'Date_to_DC1' : 'min',
            'PO_Qty_to_DC2': 'sum',
            'Date_to_DC2' : 'min',
            'PO_Qty_to_DC4': 'sum',
            'Date_to_DC4' : 'min'
        }).reset_index()  # Reset index to keep grouping columns as regular columns

        # Rename the columns to match the desired output
        pivot_data.rename(columns={
            'Date_to_DC1': 'Min_del_date_to_DC1',
            'Date_to_DC2': 'Min_del_date_to_DC2',
            'Date_to_DC4': 'Min_del_date_to_DC4'
        }, inplace=True)
        
        
    except Exception as e:
        print(f"An error occurred while connecting to Access: {e}")
        return
    finally:
        # Ensure the connection is closed after processing
        if conn is not None:
            conn.close()
            print("Connection to Access database closed.")


    # Write the cleaned data with new calculations to the Excel file
    try:
        with pd.ExcelWriter(save_path, mode='w') as writer:  # Use mode 'w' to write new data without overwriting
            pivot_data.to_excel(writer, sheet_name='Pivot Import', index=False)
            cleaned_data.to_excel(writer, sheet_name='cleaned data', index=False)
        print(f"Cleaned data saved to: {save_path}")
    except Exception as e:
        print(f"An error occurred while saving the file: {e}")

# Run the function
clean_and_groupby_file_PO_pending()


File is available. Proceeding the processes


C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_12708\2002099571.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns.rename(columns={'Name' : 'Product Name',
C:\Users\Thanawit C\AppData\Local\Temp\ipykernel_12708\2002099571.py:84: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  access_data = pd.read_sql(query, conn)


Connection to Access database closed.
Cleaned data saved to: D:\Data for Stock Report\cleaned_PO_pending_import_party.xlsx
